<a href="https://colab.research.google.com/github/v5v5/trading/blob/main/trade_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

URL to download quotes EURUSD

https://query1.finance.yahoo.com/v7/finance/download/EURUSD=X?period1=1262304000&period2=1619654400&interval=1d&events=history&includeAdjustedClose=true

In [281]:
import pandas as pd
from urllib.request import Request, urlopen
from io import StringIO
import datetime
import time
import random

In [239]:
# dt_from = datetime.date(2000,1,1)
dt_from = datetime.date(2019,1,1)
ut_from = str(int(time.mktime(dt_from.timetuple())))
ut_from

'1546300800'

In [240]:
dt_to = datetime.date.today()
ut_to = str(int(time.mktime(dt_to.timetuple())))
ut_to

'1619654400'

In [241]:
# url='https://query1.finance.yahoo.com/v7/finance/download/EURUSD=X?period1=1262304000&period2=1619654400&interval=1d&events=history&includeAdjustedClose=true'
url='https://query1.finance.yahoo.com/v7/finance/download/EURUSD=X?period1=' + ut_from + '&period2=' + ut_to + '&interval=1d&events=history&includeAdjustedClose=true'

In [242]:
headers={'User-Agent': 'Mozilla/5.0'}

In [243]:
req = Request(url, headers=headers)

In [244]:
page = urlopen(req)
# page = urlopen(url)

In [245]:
page = urlopen(url)

In [246]:
content = page.read()

In [247]:
text = content.decode('UTF-8')

In [248]:
print(text[:300])

Date,Open,High,Low,Close,Adj Close,Volume
2019-01-01,1.149425,1.155001,1.146500,1.149306,1.149306,0
2019-01-02,1.146132,1.149700,1.134572,1.146171,1.146171,0
2019-01-03,1.131734,1.140914,1.131734,1.131811,1.131811,0
2019-01-04,1.139095,1.141774,1.134816,1.139108,1.139108,0
2019-01-07,1.141292,1.1474


In [249]:
data = StringIO(text)

In [250]:
df1 = pd.read_csv(data, sep=',', header='infer')

In [251]:
df1.tail(7)

,Date,Open,High,Low,Close,Adj Close,Volume
601,2021-04-21,1.203254,1.204529,1.200000,1.203239,1.203239,0.0
602,2021-04-22,1.203818,1.207001,1.200394,1.203775,1.203775,0.0
603,2021-04-23,1.201793,1.207799,1.201389,1.201475,1.201475,0.0
604,2021-04-26,1.208956,1.211827,1.206374,1.208985,1.208985,0.0
605,2021-04-27,1.207921,1.209190,1.205793,1.208167,1.208167,0.0
606,2021-04-28,1.208854,1.210214,1.205764,1.209031,1.209031,0.0
607,2021-04-29,1.212856,1.215510,1.210800,1.212562,1.212562,0.0


In [252]:
df1['Date'] = pd.to_datetime(df1['Date'], errors='coerce')

In [253]:
df1['DayOfWeek'] = df1['Date'].dt.dayofweek

In [254]:
df1.tail(7)

,Date,Open,High,Low,Close,Adj Close,Volume,DayOfWeek
601,2021-04-21,1.203254,1.204529,1.200000,1.203239,1.203239,0.0,2
602,2021-04-22,1.203818,1.207001,1.200394,1.203775,1.203775,0.0,3
603,2021-04-23,1.201793,1.207799,1.201389,1.201475,1.201475,0.0,4
604,2021-04-26,1.208956,1.211827,1.206374,1.208985,1.208985,0.0,0
605,2021-04-27,1.207921,1.209190,1.205793,1.208167,1.208167,0.0,1
606,2021-04-28,1.208854,1.210214,1.205764,1.209031,1.209031,0.0,2
607,2021-04-29,1.212856,1.215510,1.210800,1.212562,1.212562,0.0,3


In [255]:
df2 = df1.drop(['High','Low', 'Adj Close', 'Volume'], axis=1)

In [256]:
df2.tail(7)

,Date,Open,Close,DayOfWeek
601,2021-04-21,1.203254,1.203239,2
602,2021-04-22,1.203818,1.203775,3
603,2021-04-23,1.201793,1.201475,4
604,2021-04-26,1.208956,1.208985,0
605,2021-04-27,1.207921,1.208167,1
606,2021-04-28,1.208854,1.209031,2
607,2021-04-29,1.212856,1.212562,3


In [257]:
df2['Close-1'] = df2['Close'].shift(1)

In [258]:
df2.tail(7)

,Date,Open,Close,DayOfWeek,Close-1
601,2021-04-21,1.203254,1.203239,2,1.204529
602,2021-04-22,1.203818,1.203775,3,1.203239
603,2021-04-23,1.201793,1.201475,4,1.203775
604,2021-04-26,1.208956,1.208985,0,1.201475
605,2021-04-27,1.207921,1.208167,1,1.208985
606,2021-04-28,1.208854,1.209031,2,1.208167
607,2021-04-29,1.212856,1.212562,3,1.209031


In [259]:
df2['%'] = (df2['Close'] - df2['Close-1']) / df2['Close-1'] * 100

In [274]:
df2.head(7)

,Date,Open,Close,DayOfWeek,Close-1,%
0,2019-01-01,1.149425,1.149306,1,NaN,NaN
1,2019-01-02,1.146132,1.146171,2,1.149306,-0.272773
2,2019-01-03,1.131734,1.131811,3,1.146171,-1.252867
3,2019-01-04,1.139095,1.139108,4,1.131811,0.644719
4,2019-01-07,1.141292,1.141044,0,1.139108,0.169958
5,2019-01-08,1.147934,1.147974,1,1.141044,0.607339
6,2019-01-09,1.145725,1.145607,2,1.147974,-0.206189


In [285]:
def decision(index):
  row = df2.loc[index]
  # print(index, row['Date'])
  return random.choice(['Buy', 'Sell'])

In [286]:
def trade():
  for index, row in df2.iterrows():
    dcs = decision(index)
    print(index, row['Date'], dcs)

In [287]:
trade()

0 2019-01-01 00:00:00 Buy
1 2019-01-02 00:00:00 Sell
2 2019-01-03 00:00:00 Buy
3 2019-01-04 00:00:00 Sell
4 2019-01-07 00:00:00 Buy
5 2019-01-08 00:00:00 Sell
6 2019-01-09 00:00:00 Sell
7 2019-01-10 00:00:00 Sell
8 2019-01-11 00:00:00 Buy
9 2019-01-14 00:00:00 Buy
10 2019-01-15 00:00:00 Sell
11 2019-01-16 00:00:00 Sell
12 2019-01-17 00:00:00 Sell
13 2019-01-18 00:00:00 Buy
14 2019-01-21 00:00:00 Sell
15 2019-01-22 00:00:00 Buy
16 2019-01-23 00:00:00 Sell
17 2019-01-24 00:00:00 Buy
18 2019-01-25 00:00:00 Sell
19 2019-01-28 00:00:00 Sell
20 2019-01-29 00:00:00 Sell
21 2019-01-30 00:00:00 Buy
22 2019-01-31 00:00:00 Buy
23 2019-02-01 00:00:00 Buy
24 2019-02-04 00:00:00 Buy
25 2019-02-05 00:00:00 Buy
26 2019-02-06 00:00:00 Sell
27 2019-02-07 00:00:00 Buy
28 2019-02-08 00:00:00 Sell
29 2019-02-11 00:00:00 Buy
30 2019-02-12 00:00:00 Sell
31 2019-02-13 00:00:00 Sell
32 2019-02-14 00:00:00 Sell
33 2019-02-15 00:00:00 Sell
34 2019-02-18 00:00:00 Buy
35 2019-02-19 00:00:00 Buy
36 2019-02-20 00:00